# Following is the code for the Project - Opening first Sports Accessories and Equipment shop in Toronto

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## Checking the existing sports shops located in toronto

In [2]:
# The code was removed by Watson Studio for sharing.

## Getting the location of Toronto City

In [3]:
#Getting co-ordinates of Toronto City
t_address = 'Toronto City, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
t_location = geolocator.geocode(t_address)
t_latitude = t_location.latitude
t_longitude = t_location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(t_latitude, t_longitude))

The geograpical coordinate of Toronto are 43.7189883, -79.44157.


## Getting the details of the existing Sports shop in Toronto City

In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 20000 # define radius
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=sport'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    t_latitude, 
    t_longitude, 
    radius, 
    LIMIT)

In [5]:
results = requests.get(url).json()

In [6]:
# Getting relevant fields from result json
exis_sport = results['response']['venues']
exist_sports = json_normalize(exis_sport)

exist_sports.shape
len(results['response']['venues'])

50

In [7]:
#Getting the categories from the result json
categories = [0 for i in range(50)]
n=0
for item in results['response']['venues']:
    try:
        categories[n]=item['categories'][0]['name']
    except:
        categories[n]=np.nan
    
    n=n+1

categories[0:10]

['Sporting Goods Shop',
 'Sporting Goods Shop',
 'Sporting Goods Shop',
 'Sporting Goods Shop',
 'Medical Center',
 'Sporting Goods Shop',
 'Sporting Goods Shop',
 'Sporting Goods Shop',
 nan,
 'Sporting Goods Shop']

In [8]:
category=pd.DataFrame(categories)
exist_sports['Category']=category[0]
exist_sports['Category'].unique()

array(['Sporting Goods Shop', 'Medical Center', nan, 'Home Service',
       'Medical School', 'Gym / Fitness Center', 'College Gym',
       'Bike Shop', 'Salon / Barbershop', 'Other Nightlife', 'Bar',
       'Office', 'Optical Shop', 'Athletics & Sports', 'Supplement Shop',
       'Sports Bar', 'Automotive Shop'], dtype=object)

In [9]:
existing_sports=exist_sports[['location.lat', 'location.lng', 'name', 'Category']]
existing_sports.head()

,location.lat,location.lng,name,Category
0,43.726996,-79.451595,Sport Chek,Sporting Goods Shop
1,43.712860,-79.399611,Sport Chek Yonge Street,Sporting Goods Shop
2,43.655531,-79.377798,Sport Chek,Sporting Goods Shop
3,43.777761,-79.343158,Sport Chek,Sporting Goods Shop
4,43.707956,-79.394012,Cleveland Clinic Canada - Sport Medicine,Medical Center


## Dropping the venues which are not sport shops

In [10]:
n=0
for item in existing_sports.Category:
    if item in ['Medical Center', np.nan, 'Home Service',
       'Medical School', 'Gym / Fitness Center', 'College Gym',
       'Salon / Barbershop', 'Bike Shop', 'Other Nightlife', 'Bar',
       'Office', 'Optical Shop', 'Supplement Shop',
       'Sports Bar', 'Automotive Shop']:
        existing_sports.drop(n,inplace=True)
        
    n=n+1
existing_sports.reset_index(drop = True, inplace=True)
existing_sports.columns=['latitude', 'longitude', 'Name', 'Category']
existing_sports.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,latitude,longitude,Name,Category
0,43.726996,-79.451595,Sport Chek,Sporting Goods Shop
1,43.712860,-79.399611,Sport Chek Yonge Street,Sporting Goods Shop
2,43.655531,-79.377798,Sport Chek,Sporting Goods Shop
3,43.777761,-79.343158,Sport Chek,Sporting Goods Shop
4,43.806104,-79.452892,Sport Chek,Sporting Goods Shop


In [11]:
## create map of existing Sport Shops in Toronto using latitude and longitude values

In [12]:

map_sport = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

# add markers to map
for lat, lng, name, category in zip(existing_sports['latitude'], existing_sports['longitude'], existing_sports['Name'], existing_sports['Category']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sport)  
    
map_sport

## lets check the venues nearby these sport shops

In [13]:
#Function to get category
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Sport Shop', 
                  'Sport Shop Latitude', 
                  'Sport Shop Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
sport_nearby_venues = getNearbyVenues(names=existing_sports['Name'],
                                   latitudes=existing_sports['latitude'],
                                   longitudes=existing_sports['longitude']
                                  )


In [15]:
sport_nearby_venues.head()

,Sport Shop,Sport Shop Latitude,Sport Shop Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sport Chek,43.726996,-79.451595,Nordstrom,43.726076,-79.449335,Clothing Store
1,Sport Chek,43.726996,-79.451595,Yorkdale Shopping Centre,43.725939,-79.451427,Shopping Mall
2,Sport Chek,43.726996,-79.451595,UNIQLO ユニクロ,43.726446,-79.450564,Clothing Store
3,Sport Chek,43.726996,-79.451595,SEPHORA,43.725470,-79.452000,Cosmetics Shop
4,Sport Chek,43.726996,-79.451595,Indigo Books & Music,43.727006,-79.451883,Bookstore


In [16]:
# Unique venues nearby existing sports shop
print("Total distinct venues near existing sport shops", len(sport_nearby_venues['Venue Category'].unique()))
sport_nearby_venues['Venue Category'].unique()
venues_count = sport_nearby_venues.groupby(['Venue Category'], as_index=False).count()
venues_count.sort_values(['Sport Shop'], ascending=False, axis=0, inplace=True)
venues_count.head()

Total distinct venues near existing sport shops 200


,Venue Category,Sport Shop,Sport Shop Latitude,Sport Shop Longitude,Venue,Venue Latitude,Venue Longitude
46,Coffee Shop,99,99,99,99,99,99
44,Clothing Store,89,89,89,89,89,89
153,Restaurant,44,44,44,44,44,44
66,Fast Food Restaurant,38,38,38,38,38,38
170,Sporting Goods Shop,35,35,35,35,35,35


### Thus we can see that we can use nearby venues to find an ideal location for sport shop 

## code to retrive neighborhoods and their surrounding venues in Toronto

In [17]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
data_req = data[0]
df = pd.DataFrame(data_req)
df.columns=['PostalCode', 'Borough', 'Neighborhood']
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], axis=0, inplace=True)
df1=df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()
# Replacing the neighborhood 'Not assigned' by Borough
n=0
for pcode, boro, neig in zip(df1.PostalCode, df1.Borough, df1.Neighborhood):
    if neig == 'Not assigned':
        df1['Neighborhood'][n]= df1['Borough'][n]
    n = n+1
df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Importing latitude and longitude of the Postal Codes

In [18]:
coord = pd.read_csv('https://cocl.us/Geospatial_data')

df2 = pd.concat([df1, coord], axis=1)
df2.drop(['Postal Code'], axis=1, inplace=True)
df_tor = df2
df_tor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

## Fetching venue details from Foursquare

In [20]:

#Function to get category
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Tor_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )



## Getting details venues for all the neighborhoods from Foursquare

In [21]:
#Function to get category
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
Tor_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )


In [23]:
# Dataframe with venue details
Tor_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [24]:
# Converting the venue to categorical values
Tor_data = pd.get_dummies(Tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tor_data['Neighborhood'] = Tor_venues['Neighborhood'] 

Tor_mean = Tor_data.groupby('Neighborhood').mean().reset_index()

Tor_mean.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga

In [25]:
# Finding top 5 venues for first 5 neighborhood
num_top_venues = 5

for hood in Tor_mean['Neighborhood'].head():
    print("----"+hood+"----")
    temp = Tor_mean[Tor_mean['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.05
1          Coffee Shop  0.05
2                  Bar  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1      Breakfast Spot  0.25
2              Lounge  0.25
3  Chinese Restaurant  0.25
4   Mobile Phone Shop  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0         Playground  0.33
1   Asian Restaurant  0.33
2               Park  0.33
3  Mobile Phone Shop  0.00
4      Movie Theater  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0  Grocery Store  0.18
1    Pizza Place  0.09
2       Pharmacy  0.09
3     Beer Store  0.09
4   Liquor Store  0.09


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2            

## Fetching the top 10 venues for each neighborhood and creating a dataframe of the same

In [26]:
tor_sorted = pd.DataFrame(columns = ['Neighborhood', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
tor_sorted['Neighborhood'] = Tor_mean['Neighborhood']

tor_sort_array = tor_sorted.values

num_top_venues = 10

row = 0
for hood in Tor_mean['Neighborhood']:
    temp = Tor_mean[Tor_mean['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp_array = temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues).values
    for n in range (0, num_top_venues):
        tor_sort_array[row][n+1] = temp_array[n][0]
    row = row+1
    
toronto_sorted = pd.DataFrame(tor_sort_array, columns=['Neighborhood', '1 Most Common Vennue', '2 Most Common Vennue', '3 Most Common Vennue', '4 Most Common Vennue', '5 Most Common Vennue', '6 Most Common Vennue', '7 Most Common Vennue', '8 Most Common Vennue', '9 Most Common Vennue', '10 Most Common Vennue' ])
toronto_sorted.head()

,Neighborhood,1 Most Common Vennue,2 Most Common Vennue,3 Most Common Vennue,4 Most Common Vennue,5 Most Common Vennue,6 Most Common Vennue,7 Most Common Vennue,8 Most Common Vennue,9 Most Common Vennue,10 Most Common Vennue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Bar,American Restaurant,Steakhouse,Hotel,Cosmetics Shop,Burger Joint,Bakery,Gym
1,Agincourt,Sandwich Place,Breakfast Spot,Lounge,Chinese Restaurant,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Asian Restaurant,Park,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Pharmacy,Beer Store,Liquor Store,Fried Chicken Joint,Sandwich Place,Coffee Shop,Fast Food Restaurant,Video Store
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pool,Sandwich Place,Dance Studio,Pub,Skating Rink,Coffee Shop,Gym,Middle Eastern Restaurant


## Using KMeans for Clustering

In [27]:

# set number of clusters
kclusters = 5

toronto_cluster = Tor_mean.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 0, 4, 4, 4,
       1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 1, 4, 4, 4, 1, 4, 4, 4, 2, 4,
       4, 4, 4, 1, 4, 1, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4,
       4, 1, 4, 4, 4, 4, 4, 1], dtype=int32)

In [28]:
# add clustering labels
toronto_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_tor

# Merging the neighborhood details
toronto_merged = toronto_merged.join(toronto_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(subset=['Cluster Labels'], axis=0, inplace=True)
toronto_merged['Cluster Labels'].astype('int')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1 Most Common Vennue,2 Most Common Vennue,3 Most Common Vennue,4 Most Common Vennue,5 Most Common Vennue,6 Most Common Vennue,7 Most Common Vennue,8 Most Common Vennue,9 Most Common Vennue,10 Most Common Vennue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4.0,Print Shop,Fast Food Restaurant,Miscellaneous Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Accessories Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Construction & Landscaping,Bar,Accessories Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Spa,Intersection,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Medical Center,Tech Startup,Pizza Place,Hotel Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Convenience Store,Korean Restaurant,Accessories Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Fried Chicken Joint,Lounge,Bank,Thai Restaurant,Athletics & Sports,Bakery,Caribbean Restaurant,Hakka Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


## Plotting the clusters

In [29]:
map_clusters = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype('int')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
# add markers for existing sport shops to map
for lat, lng, name, category in zip(existing_sports['latitude'], existing_sports['longitude'], existing_sports['Name'], existing_sports['Category']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
map_clusters

# Thus existing sport shops (marked in black) overlap the cluster label 4 (marked in orange). Lets further analyze each of these clusters
## Analyzing Cluster==0 

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1 Most Common Vennue,2 Most Common Vennue,3 Most Common Vennue,4 Most Common Vennue,5 Most Common Vennue,6 Most Common Vennue,7 Most Common Vennue,8 Most Common Vennue,9 Most Common Vennue,10 Most Common Vennue
91,Etobicoke,0.0,Baseball Field,Accessories Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Museum
97,North York,0.0,Baseball Field,Accessories Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Museum


## These are clusters with baseball fields and small shops. This is good location for Baseball theme sport shop preferably in tie up with the baseball fields, as these areas are not visited for shopping but mostly for playing baseball. 

## Analyzing Cluster==1

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1 Most Common Vennue,2 Most Common Vennue,3 Most Common Vennue,4 Most Common Vennue,5 Most Common Vennue,6 Most Common Vennue,7 Most Common Vennue,8 Most Common Vennue,9 Most Common Vennue,10 Most Common Vennue
23,North York,1.0,Park,Convenience Store,Bank,Bar,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant
25,North York,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Accessories Store,Miscellaneous Shop,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
30,North York,1.0,Airport,Park,Other Repair Shop,Accessories Store,Miscellaneous Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
40,East York,1.0,Convenience Store,Coffee Shop,Park,Accessories Store,Miscellaneous Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
44,Central Toronto,1.0,Park,Bus Line,Swim School,Accessories Store,Mobile Phone Shop,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
50,Downtown Toronto,1.0,Park,Trail,Playground,Accessories Store,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
64,Central Toronto,1.0,Park,Jewelry Store,Sushi Restaurant,Trail,Accessories Store,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
72,North York,1.0,Pizza Place,Park,Pub,Japanese Restaurant,Mediterranean Restaurant,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Moving Target
74,York,1.0,Park,Market,Fast Food Restaurant,Pharmacy,Women's Store,Hockey Arena,Miscellaneous Shop,Movie Theater,Motel,Monument / Landmark
79,North York,1.0,Basketball Court,Construction & Landscaping,Bakery,Park,Deli / Bodega,Accessories Store,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant


# This cluster mainly has parks and is frequented to visit parks and not for shopping. So these clusters are not ideal  for a Sport Shop
## Analyzing Cluster==2

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1 Most Common Vennue,2 Most Common Vennue,3 Most Common Vennue,4 Most Common Vennue,5 Most Common Vennue,6 Most Common Vennue,7 Most Common Vennue,8 Most Common Vennue,9 Most Common Vennue,10 Most Common Vennue
5,Scarborough,2.0,Playground,Accessories Store,Miscellaneous Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
14,Scarborough,2.0,Playground,Asian Restaurant,Park,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
48,Central Toronto,2.0,Tennis Court,Playground,Miscellaneous Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant


# These ares have playgrounds and will be ideal for sport shop accessories. However these are not shopping places and hence a small store is good in these areas.
## Analyzing Cluster==3

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1 Most Common Vennue,2 Most Common Vennue,3 Most Common Vennue,4 Most Common Vennue,5 Most Common Vennue,6 Most Common Vennue,7 Most Common Vennue,8 Most Common Vennue,9 Most Common Vennue,10 Most Common Vennue
94,Etobicoke,3.0,Bank,Accessories Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Museum


# This area is not shopping area and hence not an ideal location for a sport shop
## Analyzing Cluster==4

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1 Most Common Vennue,2 Most Common Vennue,3 Most Common Vennue,4 Most Common Vennue,5 Most Common Vennue,6 Most Common Vennue,7 Most Common Vennue,8 Most Common Vennue,9 Most Common Vennue,10 Most Common Vennue
0,Scarborough,4.0,Print Shop,Fast Food Restaurant,Miscellaneous Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Accessories Store
1,Scarborough,4.0,Construction & Landscaping,Bar,Accessories Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
2,Scarborough,4.0,Spa,Intersection,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Medical Center,Tech Startup,Pizza Place,Hotel Bar
3,Scarborough,4.0,Coffee Shop,Convenience Store,Korean Restaurant,Accessories Store,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
4,Scarborough,4.0,Fried Chicken Joint,Lounge,Bank,Thai Restaurant,Athletics & Sports,Bakery,Caribbean Restaurant,Hakka Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
6,Scarborough,4.0,Discount Store,Playground,Department Store,Coffee Shop,Molecular Gastronomy Restaurant,Museum,Moving Target,Movie Theater,Motel,Monument / Landmark
7,Scarborough,4.0,Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Park,Metro Station,Intersection,Bus Station,Accessories Store,Modern European Restaurant
8,Scarborough,4.0,Motel,American Restaurant,Music Store,Moving Target,Movie Theater,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Accessories Store
9,Scarborough,4.0,College Stadium,Café,General Entertainment,Skating Rink,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant
10,Scarborough,4.0,Indian Restaurant,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Mobile Phone Shop,Movie Theater,Motel,Monument / Landmark,Molecular Gastronomy Restaurant


# These neighborhoods have good number of shopping centres and playgrounds and is ideal for opening a new Sport Shop. Further parameters like rental, space availability etc will zeroize the location of the sport shop